[Here](https://stackoverflow.com/questions/20186344/importing-an-ipynb-file-from-another-ipynb-file) i obtained the way to import a jupyter notebook inside another one.

In the column "Name" of HMXB_cat_Neumann catalog, on row 175 to 192 of kim_persistent, i have elements i want to split by simple spaces, but the names are in the second column once split the columns. In third place, in kim_transient between 304 and 367 i do have names in the "Names" column. Also, in malacaria_persistent and malacaria_transient the names of the objects are in the column named "Sources", while in the v2023-09-Fortin catalog the names are in "Main_ID" column. This is useful to know whether all the objects are in the four catalogs. 

### kim_transient Catalog with it's names (Here the Kim catalog is divided in two: transient and persistent, as malacaria catalog is).

In [1]:
import pandas as pd
import numpy as np
import re
from IPython.display import Image, display
from mpl_toolkits.basemap import Basemap
import matplotlib.image as mpimg
from astropy import units as u
import matplotlib.pyplot as plt
from astropy.coordinates import SkyCoord
from astropy.coordinates import Angle
import seaborn as sns


nombre_excel = "HMXB_catalogs.xlsx"

kim_transient = pd.read_excel(nombre_excel, sheet_name="kim_transient")
columna_kim_t = kim_transient.iloc[302:367, 0]
df_kim_t = columna_kim_t.str.split(expand=True)
nombres_kim_t = df_kim_t[1] + " " + df_kim_t[2]

C:\Users\Acer\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Acer\anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\Acer\anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
C:\Users\Acer\anaconda3\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Kim_persistent Catalog with it's names.

In [2]:
kim_persistent = pd.read_excel(nombre_excel, sheet_name="kim_persistent")
columna_kim_p = kim_persistent.iloc[173:192, 0]
df_kim_p = columna_kim_p.str.split(expand=True)
nombres_kim_p = df_kim_p[1] + " " + df_kim_p[2]

### HMXB_cat_Neumann Catalog with it's names.

In [3]:
cat_neuman = pd.read_excel(nombre_excel, sheet_name="HMXB_cat_Neumann")
nombres_neuman = cat_neuman["Name"]


nombre_excel_2 = 'HMXB_cat.xlsx'
cat_neuman_2 = pd.read_excel(nombre_excel_2, sheet_name="HMXB_cat")

### Malacaria_persistent Catalog with it's names.

In [4]:
malacaria_persistent = pd.read_excel(nombre_excel, sheet_name="malacaria_persistent")
columna_sources = malacaria_persistent["Source"]
nombres_malacaria_persistent = columna_sources.iloc[1:].str.replace(r"\s*\(.*?\)", "", regex=True)

### Malacaria_transient Catalog with it's names.

In [5]:
malacaria_transient = pd.read_excel(nombre_excel, sheet_name="malacaria_transient")
columna_sources_transient = malacaria_transient["Source"]
nombres_malacaria_transient = columna_sources_transient.iloc[1:].str.replace(r"\s*\(.*?\)", "", regex=True)

### v2023_09_Fortin Catalog with it's names. 

In [6]:
v2023_09_Fortin = pd.read_excel(nombre_excel, sheet_name="v2023-09_Fortin")
columna_main_id = v2023_09_Fortin["Main_ID"]
nombres_fortin = columna_main_id.str.replace(r"\s*\(.*?\)", "", regex=True)

### As we will see later, there is no binary system simultaneously on six "catalogs", because there aren't 6 at all (just four catalogs, which two of them are divided by two: transient and persistent).

In [7]:
todas_las_nombres = pd.Series(
    list(nombres_kim_t) + list(nombres_kim_p) + list(nombres_neuman) + 
    list(nombres_malacaria_persistent) + list(nombres_malacaria_transient) + 
    list(nombres_fortin)
)

conteo_nombres = todas_las_nombres.value_counts()
elementos_en_dos_o_mas_catalogos = conteo_nombres[conteo_nombres >= 2]

### Now we gonna see the distribution in Neumann Catalog for objects of it that are present in 1, 2, 3 or all of 4 catalogs in a RA (right ascension) versus DEC (declination).

In [8]:
kim_combined = pd.concat([
    pd.read_excel(nombre_excel, sheet_name="kim_transient").iloc[302:367, 0],
    pd.read_excel(nombre_excel, sheet_name="kim_persistent").iloc[173:192, 0]
]).str.split(expand=True).apply(lambda x: ' '.join(x[1:3]), axis=1)

malacaria_combined = pd.concat([
    pd.read_excel(nombre_excel, sheet_name="malacaria_persistent")["Source"].iloc[1:].str.replace(r"\s*\(.*?\)", "", regex=True),
    pd.read_excel(nombre_excel, sheet_name="malacaria_transient")["Source"].iloc[1:].str.replace(r"\s*\(.*?\)", "", regex=True)
])

todos_los_nombres = pd.Series(
    list(kim_combined) + list(malacaria_combined) + list(nombres_neuman) + list(nombres_fortin)
)

conteo_nombres = todos_los_nombres.value_counts()

elementos_en_cuatro_catalogos = conteo_nombres[conteo_nombres >= 4]
elementos_en_cuatro_catalogos = elementos_en_cuatro_catalogos.index.tolist()

In [9]:
common_names = set(nombres_neuman).intersection(set(nombres_fortin))
df_common_names = pd.DataFrame(common_names, columns=['Nombres Comunes'])

cols_f = ['Main_ID', 'Spectype', 'Mo', 'Period', 'Eccentricity', 'Spin_period','Distance','Class']
cols_n = ['Name', 'SpType', 'Mean_Mass', 'Teff','N_H','Max_Soft_Flux', 'Min_Soft_Flux', 'Max_Hard_Flux','Min_Hard_Flux']#'Ppulse', 'Porb'
v2023_09_Fortin_comunes = v2023_09_Fortin[v2023_09_Fortin['Main_ID'].isin(common_names)][cols_f]
cat_neuman_comunes = cat_neuman[cat_neuman['Name'].isin(common_names)][cols_n]

v2023_09_Fortin_comunes = v2023_09_Fortin_comunes.rename(columns={'Main_ID': 'Nombre'})
cat_neuman_comunes = cat_neuman_comunes.rename(columns={'Name': 'Nombre'})

df_final = pd.merge(v2023_09_Fortin_comunes, cat_neuman_comunes, on='Nombre')

In [10]:
df_final['Mean_Soft_Flux'] = df_final[['Min_Soft_Flux', 'Max_Soft_Flux']].mean(axis=1)

df_final['Mean_Hard_Flux'] = df_final[['Min_Hard_Flux', 'Max_Hard_Flux']].mean(axis=1)

df_final['Hardness'] = df_final['Mean_Hard_Flux'] / df_final['Mean_Soft_Flux']

In [11]:
df_final

,Nombre,Spectype,Mo,Period,Eccentricity,Spin_period,Distance,Class,SpType,Mean_Mass,Teff,N_H,Max_Soft_Flux,Min_Soft_Flux,Max_Hard_Flux,Min_Hard_Flux,Mean_Soft_Flux,Mean_Hard_Flux,Hardness
0,IGR J00370+6122,BN0.7 Ib,22.0,15.66490,0.4800,674.0000,3401.0,sg,BN0.5II-III / BN0.7Ib,NaN,15411.1,11.820,95.5227,0.1253,18.420,8.008,47.82400,13.2140,0.276305
1,gam Cas,B0.5IVpe,13.0,203.37100,0.2600,NaN,NaN,Be,B0.5IVpe,NaN,NaN,NaN,274.0580,157.7170,83.440,83.440,215.88750,83.4400,0.386498
2,2S 0114+650,B1Iae,16.0,11.59830,0.1800,10008.0000,4475.0,sg,B1Iae,NaN,14131.5,NaN,130.8930,101.1000,155.400,155.400,115.99650,155.4000,1.339696
3,IGR J01363+6610,B1Ve,12.5,159.00000,NaN,NaN,5816.0,Be,B1Ve,NaN,NaN,NaN,0.1174,0.1174,NaN,NaN,0.11740,NaN,NaN
4,RX J0146.9+6121,B1IIIe,9.6,330.00000,NaN,1407.4000,2751.0,Be,B1III-Ve,NaN,NaN,3.908,105.5000,25.4422,NaN,NaN,65.47110,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,GRO J2058+42,O9.5-B0IV-Ve,18.0,55.00000,NaN,195.2500,8861.0,Be,O9.5-B0IV-Ve,NaN,31462.3,NaN,167.9000,167.9000,NaN,NaN,167.90000,NaN,NaN
95,SAX J2103.5+4545,B0Ve,17.5,12.66536,0.4055,358.6100,6218.0,Be,B0Ve,NaN,29372.9,28.000,111.6000,0.2226,97.955,68.150,55.91130,83.0525,1.485433
96,Cep X-4,B1-B2Ve,10.8,20.85000,NaN,65.3508,7446.0,Be,B1-B2Ve,NaN,NaN,7.115,470.1000,0.6595,NaN,NaN,235.37975,NaN,NaN
97,1H 2202+501,Be,NaN,NaN,NaN,NaN,1116.0,Be,B3e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Task 0 (non-interactive_part_proyect):

Make the following graphics:
- venn's diagram for all catalogs
- right ascension vs declination for neumann catalog (4 catalogs/vs no)
- right ascension vs declination for neumann catalog (1,2,3 and 4 catalogs)
- Longitud vs Galactic latitud for  neumann catalog
- ilustration of the galactic coordinates
- Hammer's Proyection of the HMXBs for Neumann catalog
- explicit & algebraic relation between distance/galactic coordinates.
- galactic distribution for Neumann Catalog
- Corbet's diagram 
- coordinates conversion
- galactic coordinates for Kim catalog
- equatorial coordinates for Kim catalog


## Task1 (done)

Check the content of the .corr method for panda arrays, and the detail of the computation performed when you use method=‘kendall'


The .corr function in python compute pairwise correlation of columns, excluding NA/null values [as it's in this page](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html). Also, the method parameter can compute the pearson, kendall or the spearman correlation coefficients. In our case, we wanna use the kendall tau because it take into account the non-linear correlations betweeen two variables. For example, as in Fortin et al 2022, the Porb and the Pspin have the next relation:

$$P_{\text{spin}} \propto P_{\text{orb}}^2$$
which can't be explained using a linear coefficient as Pearson/Spearman correlation coefficients.

## Task2 (done)

Remove Ppulse (or spin_period) and Porb (or Period). As i explained before, the pulse period (also called spin period) of a neutron star is strongly correlated with the Orbital Period, so the removal of one of them is loggical. 

## Task3:

**Figure out why there are disagreements between Porb and Period (from 2 different catalogs, right?).** First of all, we have to define what Ppulse and Porb are in the papers. Ppulse comes from Neumann Catalog, while Spin_period comes from Fortin Catalog. 

In Neumann Catalog, Ppulse is: "Pulsation Period (spin) of the binary (with NS) in seconds if determined". 

Authors made the Fortin Catalog from a mix between existing and new data about HXMB's. For example, in Fortin W63 X-1 have a Spin_period of 36 seconds, while in Neuman it has a 36 seconds ppulse. Both Catalogs come from a mix between other catalogs, so it's difficult to answer. I noticed that the differences became from numbers like 36.0 and 36.01 (the difference is minimal!).

## Task5 (non_interactive_part_proyect)

Look for a way to plot 2 dendrograms for >0 and <0 correlations. Done.

## Task6 (non_interactive_part_proyect):

Plot histogram of soft_flux ((geometric?) average between min and max) and compare to Fortin fig.4. I made it without an outlier.

## Task7 (non_interactive_part_proyect)


Recompute Kendall and dendrogram with mean_soft_flux, mean_hard_flux and hardness (which is the ratio between mean_hard_flux and mean_soft_flux). To understand the meaning of hardness, check Fig.2 from https://arxiv.org/pdf/2410.21456 and Fig.10 from https://www.aanda.org/articles/aa/pdf/2023/06/aa45708-22.pdf.

## Task8 (in progress):

Deduce stellar properties from spectral type (in particular the mass and, if possible, radius, luminosity and temperature). FYI, the standard procedure to deduce the mass of a star from 3 different methods is (here, just focus on getting the spectral-type - mass):
1. Spectral-type - mass: it is obtained by computing the mean and standard deviation of the evolutionary masses of apparently-single stars with similar spectral types. The masses of these stars can be computed for instance from their measured physical properties (e.g., luminosity, effective temperature, surface gravity) as input in the [BONNSAI Bayesian tool](https://www.astro.uni-bonn.de/stars/bonnsai/) ([Shneider+14](https://ui.adsabs.harvard.edu/abs/2014A&A...570A..66S/abstract)), which estimates the evolutionary mass using single-star evolution tracks (see [Schneider+18](https://ui.adsabs.harvard.edu/abs/2018A&A...618A..73S/abstract) for an example). 
2. Evolutionary mass: it is obtained by inputting the values of stellar luminosity and effective temperature in the BONNSAI tool.
3. Spectroscopic mass: it is obtained from the surface gravity (deduced from the width of the absorption lines in the spectrum) and stellar radius derived from spectroscopy. The stellar parameters and abundances are derived from a spectroscopic analysis (e.g. with the CMFGEN model atmosphere code, see section “spectral analysis” in supplementary information [Shenar+22](https://ui.adsabs.harvard.edu/abs/2022NatAs...6.1085S/abstract) for more details).

Links to papers and catalogs

[Neumann+23
HMXB_cat](http://astro.uni-tuebingen.de/~xrbcat/HMXBcat.html)



[Fortin+23
v2023-09](https://ui.adsabs.harvard.edu/abs/2023A&A...671A.149F/abstract)



[Kim+23
kim_*](https://ui.adsabs.harvard.edu/abs/2023ApJS..268...21K/abstract) (bottom of tables 2 and 3 in appendix)



[Malacaria+20
malacaria](https://ui.adsabs.harvard.edu/search/q=author%3A%22%5Emalacaria%2Cc%22&sort=date%20desc%2C%20bibcode%20desc&p_=0) (bottom of table 1)

## Task 9 (interactive_part_proyect)

1. The correlation between NS spin and orbital period doesn't appear in your Kendall matrix, while it does in the Corbet diagram but only for systems with a Be start. Could you set up a switch such as you can perform the same steps (Kendall+dendrogram) either for all stars (as you already did) or separately for Be stars on one hand, and for other types of stars on the other hand? 

## Task 10 (in progress)

2. A more involving idea. In the database you're using, an important information is missing : the stellar wind properties (mass loss rate and terminal wind speed). It's especially important when the donor star is a blue supergiant (i.e. not a Be star). The main reason it's missing is that we don't have *that* many systems where it's measured accurately. Yet, in this paper (https://ui.adsabs.harvard.edu/abs/2020A%26A...634A..49H/abstract), you'll find data for 6 systems which should be in the catalogs you're using. Could you check it? I know that it's very few objects, but we could extrapolate the values to other types of similar spectral type. Could you quickly check whether these 2 new parameters correlate with the others? 

## Task 11 (interactive_part_proyect)

3. Linked to the previous point: it'd be useful in your jupyter notebook if you had an interface where we can easily select two parameters to be plotted for all the systems, to verify whether the Kendall coefficients make sense. Possibly, you could plot two more parameters (e.g. as dots of different size and color).

## Task 12 (interactive_part_proyect)

Could you also plot a [Venn diagram](https://en.wikipedia.org/wiki/Venn_diagram) of the systems, with each block representing a parameter (e.g. the ~6 ones I sent last time and that you use for the Kendall tau matrix)? Something along [these lines](https://upload.wikimedia.org/wikipedia/commons/5/56/6-set_Venn_diagram_SMIL.svg) for instance? It would serve to know which is the fraction of systems where all the parameters are known.



## Task 13 (interactive_part_proyect):

Venn diagram from Fortin and Neumann’s catalogs => 17 systems w/ known:
· spectral type
· stellar mass 	
· orbital period
· eccentricity
· spin period
· distance

+ 6 where only the stellar mass is missing (but could be deduced from spectral type)
+ 8 where only the eccentricity is missing
+ 5 where only the spin period is missing (potential black holes!)

task: check by hand the 17 and the 8 to make sure that it works fine

## Past task:

keywords to look for on Google are “catalog” “Be stars” “spectral type” “mass” “relation"

## Task 14 (non_interactive_part_proyect): 

feature to get the list of systems which have (i) all parameters (17 systems), (ii) eccentricity missing, (iii) eccentricity and orbital period missing, … ask input to user (which missing parameters do you want) and give the list with names and parameters.

repeat w/ log scale to reproduce Fortin’s fig.4 and check units. Work w/ updated Neumann’s catalog (from http://astro.uni-tuebingen.de/~xrbcat/HMXBcat.html) and read 2.2 of their paper to see which instrument corresponds to which energy range. For instance, to reproduce Fortin’s fig.4, use Swift/BAT. To convert fluxes to luminosity, account for distance estimates. Check method in last paragraph of Fortin’s section 3 to combine Swift/BAT fluxes w/ Gaia distances to get luminosity. To convert a flux F you measure from Earth of a body at distance D to an intrinsic luminosity L:

L=F*(4\pi D^2)

Rk: technically, you should also account for absorption along LOS by the ISM.


Exo: given F_min=1E-13 erg/s/cm2 detectable (w/ Swift’s sensitivity according to https://heasarc.gsfc.nasa.gov/W3Browse/swift/swbat105m.html), what’s the maximum distance at which I can detect a source as a function of its luminosity?

Fortin Quote:

"First, we use
the distances in Table A.1, which were queried in Bailer-Jones
et al. (2021) using the Gaia DR3 positions. Then, we query the
Swift 2SXPS catalogue using their available Swift identifiers, and
fetch the value of the unabsorbed flux in the 0.3–10 keV band
(column apec_flux_b)" While in the [web site](https://www.swift.ac.uk/2SXPS/) is the Swift Catalog from [Evans et al 2020]( https://ui.adsabs.harvard.edu/abs/2020ApJS..247...54E/abstract).

## Task 14 (interactive_part_proyect): 

feature to get the list of systems which have (i) all parameters (17 systems), (ii) eccentricity missing, (iii) eccentricity and orbital period missing, … ask input to user (which missing parameters do you want) and give the list with names and parameters.

## Task 15:

repeat w/ log scale to reproduce Fortin’s fig.4 and check units. Work w/ updated Neumann’s catalog (from http://astro.uni-tuebingen.de/~xrbcat/HMXBcat.html) and read 2.2 of their paper to see which instrument corresponds to which energy range. For instance, to reproduce Fortin’s fig.4, use Swift/BAT. To convert fluxes to luminosity, account for distance estimates. Check method in last paragraph of Fortin’s section 3 to combine Swift/BAT fluxes w/ Gaia distances to get luminosity. To convert a flux F you measure from Earth of a body at distance D to an intrinsic luminosity L:

L=F*(4\pi D^2)

Rk: technically, you should also account for absorption along LOS by the ISM.

Exo: given F_min=1E-13 erg/s/cm2 detectable (w/ Swift’s sensitivity according to https://heasarc.gsfc.nasa.gov/W3Browse/swift/swbat105m.html), what’s the maximum distance at which I can detect a source as a function of its luminosity?

Fortin Quote:

"First, we use
the distances in Table A.1, which were queried in Bailer-Jones
et al. (2021) using the Gaia DR3 positions. Then, we query the
Swift 2SXPS catalogue using their available Swift identifiers, and
fetch the value of the unabsorbed flux in the 0.3–10 keV band
(column apec_flux_b)" While in the [web site](https://www.swift.ac.uk/2SXPS/) is the Swift Catalog from [Evans et al 2020]( https://ui.adsabs.harvard.edu/abs/2020ApJS..247...54E/abstract). Besides this, [Gaia DR3](https://www2.mpia-hd.mpg.de/~calj/gedr3_distances.html) is in [Bailer-Jones et al 2021](https://arxiv.org/pdf/2012.05220), which sent me to [this.](https://vizier.cds.unistra.fr/viz-bin/VizieR-3)

When do you started being an astronomer? Did you always have interest in the pulsars like bynary systems?

In the first place, why do you decided to use those 4 catalogs and no other? it have any relation with the completeness of any of them (i.e. the quantity of systems or the amount of parameters for them)?

In the case of the distances from GAIA DR3 do i have to query with the GAIA DR3 ID from neumann catalog?

In the case I use the BAT fluxes from actualized Neumann Catalog, do i take the BAT_min_flux or the BAT_max_flux? 

In the case I use the Swift Fluxes, Do I have to make a query to match all the objects in the Neumann+Fortin Catalog, right?

How do we usually select the priors for the observables in the prediction of the models (i.e. parameters) in the bayesian Tool?

Have you used the BONNSAI tool? For which cases? How do you use the BONNSAI tool for multiple systems simultaneously?

Which of the three kinds of masses I'm supposed to obtain from the catalogs (as the observables from Fortin+Neumann catalog), since i don't know the uncertainties of the effective temperature neither the uncertainties from the surface gravity (neither the mean of this), neither luminosities? 

Or do I have to obtain the luminosities from the Flux and then use them to predict the mass with BONNSAI?

When we talk about the luminosity of the HMXBs, we are talking about the luminosity of both objects like one object? Or we are talking about the Companion of the Neutron star alone?

Why in the Fig 5 Of Fortin the quantity of detected or identified spectral type increases? e.g. Be and sg.?

I was thinking, it's the imputing of the median an inadequate form of using the data for the kendall correlation matrix and the Dendograms because of the definition of the median when there are a even amount of data for a determined parameter as the effective temperature. Right?

It's possible to use the Bootstrap technique in some cases to determine the kendall Tau's coefficient with it's uncertainties? (e.g. with replacement)

How can i take into account the absortion in the line of sight?

There have to be a maximum distance because its contrary the flux have a minimum? When we say that a pulsar have a minimum flux, we are talking about a specific band in the Swift sensitivity?







Monday 16-12-2024

1) What's the definition of near-infrared?
2) "Accretion through a Be disk is much more efficient at transporting angular momentum than via wind. The spin of the compact object spin is therefore correlated to the orbital period in BeHMXBs, but not in sgHMXBs" Dicho de otra manera, aquí se establece un silogismo. 

## Old task (interactive_part_proyect): 

interactive Kendall and dendrogram with switch to choose to work with Be only, Sg only or both (see https://ipywidgets.readthedocs.io/en/8.1.5/examples/Using%20Interact.html)

## New task (interactive_part_proyect): 

give the possibility to choose 2 different parameters (among the 6) and compute the correlation coefficient from all the systems where at least these 2 parameters are known. 

# Por hacer (todo en interactive_part_proyect). 

Básicamente lo que tengo que hacer es obtener lo siguiente, para ser más eficiente:
- usar el catalogo Neumann actualizado, se llama HMXB_cat.xlsx, hacer un grafico de la luminosidad, utilizando swift/bat flujos con las distancias de gaia, que estan en fortin y neumann. 

- Deduce stellar properties from spectral type (in particular the mass and, if possible, radius, luminosity and temperature). FYI, the standard procedure to deduce the mass of a star from 3 different methods is (here, just focus on getting the spectral-type - mass): Spectral-type - mass: it is obtained by computing the mean and standard deviation of the evolutionary masses of apparently-single stars with similar spectral types. The masses of these stars can be computed for instance from their measured physical properties (e.g., luminosity, effective temperature, surface gravity) as input in the BONNSAI Bayesian tool (Shneider+14), which estimates the evolutionary mass using single-star evolution tracks (see Schneider+18 for an example).

- A more involving idea. In the database you're using, an important information is missing : the stellar wind properties (mass loss rate and terminal wind speed). It's especially important when the donor star is a blue supergiant (i.e. not a Be star). The main reason it's missing is that we don't have that many systems where it's measured accurately. Yet, in this paper (https://ui.adsabs.harvard.edu/abs/2020A%26A...634A..49H/abstract), you'll find data for 6 systems which should be in the catalogs you're using. Could you check it? I know that it's very few objects, but we could extrapolate the values to other types of similar spectral type. Could you quickly check whether these 2 new parameters correlate with the others?


# Ya hechos:
- dar la posibilidad de graficar dos o hasta tres parametros cualesquiera dependiendo de la elección del usuario.
- dar los sistemas en los cuales faltan/hay todos los parametros dependiendo de los que él elija.

- el grafico de la distancia maxima en funcion de la luminosidad (non_interactive_part_proyect)
- elegir 1 o mas parametros y decirme en cuales sistemas los parametros no son conocidos (uno o más son desconocidos simultaneamente)
- poder elegir 2 parametros de entre los seis y calcular el coeficiente de correlacion de todos los sistemas donde se conocen dichos parametros.


# Por pensar 

- averiguar si es posible usar bootstrap/MCMC para conseguir incertidumbres de los parametros y así usarlos en bonsai. 

# Questions (December 30, 2024):

- It's fine that the regime of the luminosities is for 14-195 keV?
- It's fine to take the aritmetic mean of the min and max flux from bat in neumann catalog?
- It's always the same to talk about the regime/energy band in the context of the X-ray energy emission? 
- Why the near infrarred it's named like that?
- Do make any sense if i want to take a Color-Magnitude diagram from the Neumann Catalog aparent magnitudes?
- Why the near infrarred/X-ray/optical bands are called the counterparts?
- How valid is it to extrapolate the terminal velocity and the mass loss rate?
- For example, is it perhaps more valid than interpolating some known value from the parameters of the HMXBs themselves, such as the mass of the compact object?
- Are we more interested in the Mass of the compact object or in the mass of the companion?
- Does the terminal velocity refer to the terminal velocity of the pulsar traveling through the medium, or is it more related to the terminal velocity of the stellar wind from the companion star?
- Which are the esential conditions to determine wether a compact object is a black hole?
- Why were the number of Be in relation to Sg companions biased in the context of the Catalogs?
- What we understand when we talk about the poblational studies?
- In a catalog of these kind, are we any time interested in some kind of time series (as in the spin up-down of the systems) or in the graphics of flux versus wavelenght?
- It's normal that the terminal velocity correlates the most with the mass loss rate in the super giant than in the Be stars like systems?
- What is MJ2000? It's like the currently calendar of astronomers? Why?
- What's the difference between the transient and Persistent HMXBs? Does it have relation with the persistent/transient accretion systems? So for example, in the Neumann and Fortin Catalogs are they transient and/or persistent HMXBs?
